In [1]:
import pandas as pd
import numpy as np

In [2]:
ratings = [i.strip().split(",") for i in open('data/ratings.csv', 'r').readlines()]

In [20]:
ratings_df = pd.DataFrame(ratings, columns = ['UserID', 'ItemID', 'Rating', 'Timestamp'], dtype = int)
ratings_df[['Rating']] = ratings_df[['Rating']].apply(pd.to_numeric)

In [12]:
ratings_df.head()

,UserID,ItemID,Rating,Timestamp
0,A2XVJBSRI3SWDI,0000031887,5.0,1383523200
1,A2G0LNLN79Q6HR,0000031887,4.0,1337990400
2,A2R3K1KX09QBYP,0000031887,2.0,1361059200
3,A19PBP93OF896,0000031887,1.0,1363824000
4,A1P0IHU93EF9ZK,0000031887,4.0,1390435200


In [32]:
R_df = ratings_df.pivot(index = 'UserID', columns ='ItemID', values = 'Rating').fillna(0)
R_df.head()

ItemID,0000031887,0000037214,0123456479,0456787283,0456808574,0456830197,0456838384,0456840532,0456844570,0456856293,...,B00006MKT8,B00006MKXV,B00006MM6V,B00006MMGZ,B00006MN51,B00006MVYA,B00006NQ0N,B00006UD17,B00006UKTW,B00006XXGO
UserID,,,,,,,,,,,,,,,,,,,,,
A0118994XV4RVRG8RBM2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0201807Q6UDM29Y8EU6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A022042136A07KJUAFO1R,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
A024581134CV80ZBLIZTZ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
A03084523KF7K8U4QWH1Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0


In [33]:
R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [38]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)

In [39]:
sigma = np.diag(sigma)

In [42]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [46]:
def recommend_products(predictions_df, userID, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.UserID == (userID)]
    #user_full = (user_data.merge(movies_df, how = 'left', left_on = 'MovieID', right_on = 'MovieID').
    #                 sort_values(['Rating'], ascending=False)
    #             )

    print('User {0} has already rated {1} products.'.format(userID, user_data.shape[0]))
    print('Recommending the highest {0} predicted ratings product not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    
    recommendations = (pd.DataFrame(sorted_user_predictions).reset_index())
    
    #recommendations = (movies_df[~movies_df['MovieID'].isin(user_full['MovieID'])].
    #     merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
    #           left_on = 'MovieID',
    #           right_on = 'MovieID').
    #     rename(columns = {user_row_number: 'Predictions'}).
    #     sort_values('Predictions', ascending = False).
    #                   iloc[:num_recommendations, :-1]
    #                  )

    return user_data, recommendations



In [52]:
already_rated, predictions = recommend_products(preds_df, 3, ratings_df, 10)

User 3 has already rated 0 products.
Recommending the highest 10 predicted ratings product not already rated.


In [50]:
sorted_user_predictions = preds_df.iloc[2].sort_values(ascending=False)

In [51]:
sorted_user_predictions

ItemID
B00006XXGO    5.000000
B0000696C6    0.000036
0456787283    0.000036
B00006MIBT    0.000036
B00006MDHL    0.000036
9882105602    0.000036
978980685X    0.000036
8511000135    0.000036
B0000507H9    0.000036
0456840532    0.000036
6181954635    0.000036
9502738349    0.000036
9861137882    0.000036
6041006041    0.000036
8921463240    0.000036
0000037214    0.000036
B00001U01I    0.000036
8866138371    0.000036
7111324579    0.000036
B00004U3KC    0.000036
9895515510    0.000036
6175001664    0.000036
5000000048    0.000036
B00001U00V    0.000036
6152004780    0.000036
B00006BSTB    0.000035
B00006MN51    0.000035
183465193X    0.000035
0456830197    0.000035
1848572972    0.000035
                ...   
B00006IAA3   -0.000083
1617160199   -0.000084
B00005JHL3   -0.000084
B00006C28H   -0.000084
7211045639   -0.000085
B000034DKV   -0.000086
B000068ZVR   -0.000088
B00006J6U2   -0.000088
B00004U3M4   -0.000089
B00001QHXY   -0.000090
B00004U3SF   -0.000091
B00006NQ0N   -0.000091
8996